In [34]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import re
from TCSP import read_stopwords_list

# English Translated

In [35]:
data=pd.read_pickle('C:/Users/forgo/Desktop/dsa4266/full_df_en_processed.pkl').reset_index()#.drop_duplicates('clean_msg')
temp=pd.read_pickle('C:/Users/forgo/Desktop/dsa4266/full_df.pkl').reset_index()
data=data.merge(temp[['doc_id','pretranslation']],how='left',on='doc_id').drop_duplicates('pretranslation').drop_duplicates('clean_msg')
# data=data[['class','processed']].drop_duplicates().rename(columns={'processed':'clean_msg'})

data=data[data['clean_msg']!=''].reset_index(drop=True)

,doc_id,class,translated,clean_msg,pretranslation
0,trec06c/data/000/007,spam,Dear person in charge (manager/finance): Hello...,dear person charge hello trading co ltd strong...,尊敬的负责人（经理／财务）：您好！ 我是深圳伟仕嘉贸易有公司：兴办贸易、物资供销，实力雄厚；...
1,trec06c/data/000/014,spam,Hello! Nice to meet you. Our company is intere...,hello nice meet company interested cooperating...,"您好！很高兴认识您。我司有意与您们合作:可长久给您们带来 巨大的效益,另因合作项目的高度自动..."
2,trec06c/data/000/005,spam,TO: Your company’s manager and finance manager...,manager finance manager hello trading co ltd b...,TO：贵公司经理、财务 您好！ 深圳市春洋贸易有限公司（东莞分公司）...
3,trec06c/data/000/013,spam,New remote-controlled airplanes are widely sol...,new airplanes widely sold many types toy airpl...,新型遥控飞机销路广 百元可办厂 玩具飞机品种繁多，但用微型小开关控制的新型能在空中自控飞翔的...
4,trec06c/data/000/021,spam,"Dear company (factory) manager, hello: Our com...",dear company factory manager hello company com...,尊敬的公司（工厂）经理负责人你好： 我公司是一家多年为外资企业代理进出口业务的公司，现有部分...
...,...,...,...,...,...
39240,trec06p/data/035/115,spam,Rent WITHOUT COMMISSION\r \r m. Avtozavodskaya...,rent without commission min foot blocks buildi...,"Аренда БЕЗ КОМИССИОННЫХ\r \r м. Автозаводская,..."
39241,trec06p/data/058/203,ham,RIFFx~ WAVEfmt + + dataS~ ~y{yw{ zuvx}}ytmjhed...,aa aa tr pf en kn la qu oka pk oz gun nom mum ...,RIFFx~ WAVEfmt      +  +    dataS~ ...
39242,trec06p/data/014/167,spam,dangerous cork blob thirteenth alliterate argu...,dangerous cork blob thirteenth alliterate argu...,dangerous cork blob thirteenth alliterate argu...
39243,trec06p/data/025/231,spam,:*.☆. o:☆';*. :*.☆. o:☆';*. :*.☆. o:☆';*. :*.☆...,largest online community history born site abs...,\r :*.☆。o:☆';*。:*.☆。o:☆';*。:*.☆。o:☆';*。:*.☆...


In [36]:
cn_word=re.compile("[\u4e00-\u9FFF]")
data['chinese']=data['pretranslation'].apply(lambda x: ''.join([word for word in x if cn_word.match(word)]))

In [37]:
data['lang']=data['chinese'].apply(lambda x: 'c' if len(x)>2 else 'e')

In [38]:
data.groupby('lang').count()

,doc_id,class,translated,clean_msg,pretranslation,chinese
lang,,,,,,
c,23583,23583,23583,23583,23583,23583
e,15662,15662,15662,15662,15662,15662


In [39]:
# en_regex = re.compile('[^a-zA-Z ]')
# d = enchant.Dict("en_US") 
# STOPWORDS = stopwords.words('english')

In [40]:
# def enPreprocess(string):
#     string=string.replace('\n',' ') #remove newline char
#     string=en_regex.sub('',string) #removes non alphabets
#     string=string.split()
#     string=[word.lower() for word in string if len(word)>1]
#     string=' '.join([word for word in string if d.check(word) and (word not in STOPWORDS)]) # split the string into list and check each word if it is in the english dictionary and longer than 1 alphabet
#     return string

In [41]:
# data['clean_msg']=data['translated'].apply(lambda x: enPreprocess(x))
# data.to_pickle('lowercase words.pkl')

In [42]:

X = data.drop(columns='class')
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=.2,random_state=42)

In [43]:
y_train.value_counts()

ham     19566
spam     5550
Name: class, dtype: int64

In [44]:
y_train=pd.concat([y_train[y_train=='ham'].sample(5528),y_train[y_train=='spam']]).sample(frac=1) #under sampling
# y_train=pd.concat([y_train[y_train=='ham'],y_train[y_train=='spam'].sample(24461,replace=True)]).sample(frac=1) #over sampling

In [45]:
X_train=X_train.loc[y_train.index]

In [46]:
X_train_e=X_train[X_train['lang']=='e']['clean_msg']
X_val_e=X_val[X_val['lang']=='e']['clean_msg']
X_test_e=X_test[X_test['lang']=='e']['clean_msg']
y_train_e=y_train.loc[X_train_e.index]
y_val_e=y_val.loc[X_val_e.index]
y_test_e=y_test.loc[X_test_e.index]

X_train_c=X_train[X_train['lang']=='c']['chinese']
X_val_c=X_val[X_val['lang']=='c']['chinese']
X_test_c=X_test[X_test['lang']=='c']['chinese']
y_train_c=y_train.loc[X_train_c.index]
y_val_c=y_val.loc[X_val_c.index]
y_test_c=y_test.loc[X_test_c.index]

X_train=X_train['clean_msg']
X_val=X_val['clean_msg']
X_test=X_test['clean_msg']

In [47]:
vect = CountVectorizer(stop_words='english')
X_train_dtm = vect.fit_transform(X_train)
X_val_dtm = vect.transform(X_val)
X_test_dtm= vect.transform(X_test)



In [48]:
tfidvect=TfidfTransformer(smooth_idf=1)
# X_train_dtm = tfidvect.fit_transform(X_train_dtm)
# X_val_dtm = tfidvect.transform(X_val)
# X_test_dtm= tfidvect.transform(X_test)

In [49]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_val_dtm)

print('translated dataset Multinomial Naive Bayes ')
print('acc',metrics.accuracy_score(y_val, y_pred_class))
print('f1',f1_score(y_val.to_list(), y_pred_class,pos_label="spam"))
# metrics.confusion_matrix(y_test, y_pred_class)

translated dataset Multinomial Naive Bayes 
acc 0.932484076433121
f1 0.8457059679767103


In [50]:
logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_dtm, y_train)
y_pred_class = logreg.predict(X_val_dtm)
y_pred_prob = logreg.predict_proba(X_val_dtm)[:, 1]

print('translated dataset log regression ')
print('acc',metrics.accuracy_score(y_val, y_pred_class))
print('validation f1',f1_score(y_val.to_list(), y_pred_class,pos_label="spam"))

translated dataset log regression 
acc 0.9511146496815287
validation f1 0.8976325441813938


# Threshhold optimise

In [51]:
a=[]
b=[]
for i in np.arange(0.7,.85,0.001):
    a.append(i)
    c=pd.Series(y_pred_prob).apply(lambda x: 'spam' if x>i else 'ham')
    b.append(f1_score(y_val.to_list(),c,pos_label="spam"))


In [52]:
thresh_hold_table=pd.DataFrame({'threshhold value':a,'f1_score':b}).sort_values('f1_score',ascending=False).head(10)
thresh_hold_table

,threshhold value,f1_score
38,0.738,0.921006
39,0.739,0.921006
37,0.737,0.921006
40,0.740,0.920950
36,0.736,0.920680
42,0.742,0.920567
41,0.741,0.920567
31,0.731,0.920410
35,0.735,0.920354
34,0.734,0.920354


In [53]:
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
opt_predicted=pd.Series(y_pred_prob).apply(lambda x: 'spam' if x>thresh_hold_table.iloc[0,0] else 'ham')
print('test f1',f1_score(y_test.to_list(), opt_predicted,pos_label="spam"))

test f1 0.9192405780674411


# Weights Interpretation

In [54]:
sample=pd.concat([data[data['class']=='ham'].sample(8666),data[data['class']=='spam']])

In [55]:
ham_count=[]
spam_count=[]
word_list=[]
temp=pd.DataFrame({'weights':logreg.coef_[0]}).reset_index().merge(pd.DataFrame({'index':vect.vocabulary_.values(),'word':vect.vocabulary_.keys()}),how='left',on='index')
for index,(word,weights) in temp[temp['weights'].apply(lambda x:abs(x)>.9)][['word','weights']].iterrows():
    counter=sample[sample['pretranslation'].apply(lambda x: word in x)]['class'].value_counts()
    try:
        ham_count.append(counter['ham'])
    except:
        ham_count.append(0)
    try:
        spam_count.append(counter['spam'])
    except:
        spam_count.append(0)
    word_list.append(word)

word_ham_spam_counter=pd.DataFrame({'word':word_list,'ham_count':ham_count,'spam_count':spam_count})

temp=temp.merge(word_ham_spam_counter,how='left',on='word')
temp[temp['ham_count']>5].sort_values('weights',ascending=True)


,index,weights,word,ham_count,spam_count
25802,25802,-2.535008,thanks,250.0,23.0
26085,26085,-2.202937,title,55.0,69.0
28507,28507,-1.705021,wrote,972.0,12.0
14354,14354,-1.629618,kind,211.0,104.0
14994,14994,-1.598436,list,788.0,213.0
19800,19800,-1.481577,produced,35.0,8.0
21822,21822,-1.371525,ribbon,10.0,5.0
21952,21952,-1.337666,robot,283.0,5.0
18830,18830,-1.279483,pine,71.0,43.0
27084,27084,-1.217968,university,44.0,13.0


# 1 model for each language

In [23]:
vect = CountVectorizer(stop_words='english')
X_train_e_dtm = vect.fit_transform(X_train_e)
X_test_e_dtm= vect.transform(X_test_e)

In [24]:

X_train_c=X_train_c.apply(lambda x: ''.join([word for word in x if cn_word.match(word)]))
X_test_c=X_test_c.apply(lambda x: ''.join([word for word in x if cn_word.match(word)]))

In [25]:
vect = CountVectorizer(tokenizer=lambda txt:[*txt],stop_words=read_stopwords_list())
X_train_c_dtm = vect.fit_transform(X_train_c)
X_test_c_dtm= vect.transform(X_test_c)

C:\Users\forgo\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\forgo\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w', '介', '代', '令', '似', '位', '余', '作', '例', '便', '候', '值', '假', '僅', '儘', '儻', '先', '免', '全', '兩', '具', '兼', '出', '切', '前', '加', '反', '受', '句', '叮', '否', '呼', '哧', '唯', '唷', '問', '啪', '啷', '喔', '單', '喻', '噠', '固', '基', '外', '天', '夫', '奈', '妨', '妳', '始', '孰', '家', '少', '尚', '巧', '幸', '庶', '徒', '循', '怕', '恰', '悉', '惟', '慢', '成', '截', '抑', '拘', '接', '換', '料', '方', '旁', '旦', '曰', '期', '果', '根', '極', '樣', '次', '止', '正', '步', '死', '毋', '沒', '況', '消', '漫', '烏', '然', '特', '猶', '獨', '甚', '登', '直', '相', 

In [26]:

nb = MultinomialNB()
nb.fit(X_train_e_dtm, y_train_e)
y_pred_e_class = nb.predict(X_test_e_dtm)


nb = MultinomialNB()
nb.fit(X_train_c_dtm, y_train_c)
y_pred_c_class = nb.predict(X_test_c_dtm)

In [27]:
print('2 model combined Multinomial Naive Bayes ')
y_pred_combined=y_pred_e_class.tolist()+y_pred_c_class.tolist()
y_actual_combined=pd.concat([y_test_e,y_test_c]).to_list()
print('acc',metrics.accuracy_score(y_actual_combined, y_pred_combined))
print('f1',f1_score(y_pred_combined,y_actual_combined,pos_label="spam"))

2 model combined Multinomial Naive Bayes 
acc 0.9416486176583004
f1 0.8746579091406678


In [28]:

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_e_dtm, y_train_e)
y_pred_e_class = logreg.predict(X_test_e_dtm)
# y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_c_dtm, y_train_c)
y_pred_c_class = logreg.predict(X_test_c_dtm)
# y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

In [29]:

print('2 model combined log regression ')
y_pred_combined=y_pred_e_class.tolist()+y_pred_c_class.tolist()
y_actual_combined=pd.concat([y_test_e,y_test_c]).to_list()
print('acc',metrics.accuracy_score(y_actual_combined, y_pred_combined))
print('f1',f1_score(y_pred_combined,y_actual_combined,pos_label="spam"))

2 model combined log regression 
acc 0.9624155943432284
f1 0.9190672153635118


In [30]:
ham_count=[]
spam_count=[]
word_list=[]
temp=pd.DataFrame({'weights':logreg.coef_[0]}).reset_index().merge(pd.DataFrame({'index':vect.vocabulary_.values(),'word':vect.vocabulary_.keys()}),how='left',on='index')
for index,(word,weights) in temp[temp['weights'].apply(lambda x:abs(x)>.8)][['word','weights']].iterrows():
    counter=sample[sample['pretranslation'].apply(lambda x: word in x)]['class'].value_counts()
    try:
        ham_count.append(counter['ham'])
    except:
        ham_count.append(0)
    try:
        spam_count.append(counter['spam'])
    except:
        spam_count.append(0)
    word_list.append(word)

word_ham_spam_counter=pd.DataFrame({'word':word_list,'ham_count':ham_count,'spam_count':spam_count})

temp=temp.merge(word_ham_spam_counter,how='left',on='word')
temp[temp['ham_count']>5].sort_values('weights',ascending=True)

,index,weights,word,ham_count,spam_count
4435,4435,-0.995960,题,1857.0,963.0
465,465,-0.953509,历,692.0,514.0
3969,3969,-0.948641,较,1139.0,855.0
1910,1910,-0.864361,校,862.0,274.0
1619,1619,-0.859695,掩,53.0,6.0
2754,2754,-0.809725,研,899.0,651.0
1339,1339,0.826265,息,510.0,1591.0
260,260,0.867502,免,243.0,1540.0
625,625,0.942715,商,321.0,2771.0
724,724,1.087136,图,256.0,644.0


In [57]:
sample[sample['pretranslation'].apply(lambda x: '寻' in x)]

,doc_id,class,translated,clean_msg,pretranslation,chinese,lang
7093,trec06c/data/062/274,ham,Being a woman is sometimes very helpless. God ...,woman sometimes helpless god gives us little t...,做女人有时挺无奈的 老天给我们选合适人的时间太少了 才刚刚开始 时间就到了 该结婚了 ble...,做女人有时挺无奈的老天给我们选合适人的时间太少了才刚刚开始时间就到了该结婚了希望你能更爱你准...,c
6706,trec06c/data/058/006,ham,Li4u?= X-Priority: 3 X-Originating-IP: [222.82...,words searching hard found place worries even ...,Li4u?= X-Priority: 3 X-Originating-IP: [222.82...,很久以来一直想通过一种方式表述自己的言语但苦苦追寻时至今日才发现还是这里得以让我无所顾忌纵然...,c
21426,trec06c/data/196/174,ham,What is a favorite? What is true love? Unless ...,favorite true love unless never talk word love...,什么是最爱？什么是真爱？除非你我从此都不在谈及“恋爱”两字，或者即将死去的人，才有资格评价自...,什么是最爱什么是真爱除非你我从此都不在谈及恋爱两字或者即将死去的人才有资格评价自己此生的爱情...,c
346,trec06c/data/003/131,ham,Sincerely looking for short-term collaborators...,sincerely looking collaborators chance make mo...,诚寻短期合作者，暑假赚点小钱的机会！ 你要做的是提供符合以下条件的信息： 1、1-5个以零散...,诚寻短期合作者暑假赚点小钱的机会你要做的是提供符合以下条件的信息个以零散公司入住为主的写字楼...,c
21402,trec06c/data/196/129,ham,☆───────────────────────────────────☆ odjfre (...,honest understand graduate student understand ...,☆─────────────────────────────────────☆ odjfre...,说老实话我刚读研究生时也不懂但我现在懂了在中国师生关系说白了就是上级和下级的关系也可以说是雇...,c
...,...,...,...,...,...,...,...
23155,trec06c/data/212/237,spam,This is a letter in HTML format! -------------...,letter format mail system free download lifeti...,这是一封HTML格式信件！ -------------VolleyMail邮件系统-----...,这是一封格式信件邮件系统免费下载终身可用您好感谢您能在百忙之中抽出时间阅读此信函首先对冒昧地...,c
23209,trec06c/data/213/200,spam,"Socorro, the friend of Socorro and meditates w...",friend meditates crank case toothache related ...,"Socorro, the friend of Socorro and meditates w...",模具估计大师寻求和作模具估价系统主要是根据塑料成品尺寸估算出塑料模具规格以及价格可进行模具估...,c
23253,trec06c/data/214/020,spam,Countless businessmen have benefited from it. ...,countless businessmen benefited tool many ente...,无数商人从中获益 众多企业必备利刃 您有新产品却不知如何推广？您建了网站却没几个人访问？...,无数商人从中获益众多企业必备利刃您有新产品却不知如何推广您建了网站却没几个人访问您做了搜索引...,c
32853,trec06p/data/070/200,spam,How to obtain high-quality overseas customers ...,obtain overseas customers orders obtain overse...,如何获取海外优质客户与订单 如何获取海外优质客户与订单 及国际商务谈判实战技巧强化训练 ...,如何获取海外优质客户与订单如何获取海外优质客户与订单及国际商务谈判实战技巧强化训练上海深圳准...,c


In [61]:
data.loc[32853,'chinese']

'如何获取海外优质客户与订单如何获取海外优质客户与订单及国际商务谈判实战技巧强化训练上海深圳准时开课主办单位华鹰企业管理咨询公司时间地点年月日上海兆安酒店时间地点年月日深圳金融培训中心费用元人包括培训费资料费两天午餐证书费以及上下午茶点等学员对象成长型出口企业的总经理海外营销部长国际贸易部经理区域市场经理主管海外业务员驻外代表以及预备外销员和其他对国际贸易感兴趣的人士课程背景中国很多非常有竞争力的企业因为不懂得如何开拓国际市场而失去了迅速做大做强的机会部分已经出口的企业也因为不懂得如何开拓国际市场而不得不依靠外贸企业间接出口但结果是产品出口了自己并没有享受到高额的利润并游离在国际市场的门外没有自己的海外客户始终受制于外贸企业有的企业已经直接出口产品但却没有找到最有利润的市场和客户仅仅学会了出口操作而没有达到出口的真正目的获取高额销售利润与此形成鲜明对比的是全球买家越来越青睐中国制造的产品纷纷开始从中国采购或者加大从中国采购的力度实践证明主动找上门来的买家比自己主动找去的买家成交率高倍以上平均首次成交所需时间只有后者的但中国的企业却不知道如何抓住这些机会如何能够让自己轻松地被海外客户找到因而坐失商机出口营销实战系列培训课程着重帮助解决中国出口商开拓国际市场的两个核心问题快速获取国际市场与买家信息和高效出口推广策略不仅准确地定位买家而且更能让买家轻松找到和优先联络自己您的七项收益准确定位目标市场和发现高利润市场迅速地找到您全球的潜在买家和合作伙伴发现竞争对手难以发现的客户独享高利润出口订单极大丰富客户数据库不断优化客户结构提高整体客户质量提高国际市场调查技能轻松获取高价值的市场信息掌握一套获取市场信息和客户情报的系统方法知己知彼百战不殆结识同行拓展人脉积累资源培训核心内容一外销启动前的准备人才方面的准备中国各类企业国际营销部门组织架构的设定及管理方式优劣对比硬件方面的准备软件方面的准备资料方面的准备其它方面的准备二掌握产品知识应该包含哪些关键内容产品知识测试题清楚自己产品的名称清楚自己产品的技术知识及卖点清楚自己产品成本构成及报价清楚相关联产品与行业的知识及名称三如何迅速了解行业宏观环境及掌握竞争对手信息行业国际市场宏观环境所包含的要素及获取办法购买现成的行业国际市场宏观环境报告的途径利用互联网手段查询制作简易行业国际市场宏观环境利用互联网查询和分析国内最主要的竞争对手